In [2]:
import os

from vllm import LLM
from vllm.config import PoolerConfig
from vllm.entrypoints.chat_utils import (
    apply_hf_chat_template,
    parse_chat_messages,
    resolve_chat_template_content_format
)

from config import settings

/opt/miniconda3/envs/llm/lib/python3.10/site-packages/pydantic/_internal/_fields.py:152: UserWarning: Field "model_weight_dir" in Settings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [1]:
import vllm 
print(vllm.__version__)

0.7.3


In [26]:
model_dir = os.path.join(
    settings.model_weight_dir, "embedding/gme-Qwen2-VL-2B-Instruct"
)
pooler_config = PoolerConfig(
    pooling_type="LAST",
    normalize=True,
    softmax=False
)

llm = LLM(
    model=model_dir,
    trust_remote_code=False,
    device="cpu",
    max_model_len=32000,
    task="embed",
    dtype="half",
    override_pooler_config=pooler_config,
    enforce_eager=True
)

INFO 02-26 00:13:21 config.py:2444] Downcasting torch.float32 to torch.float16.
WARNING 02-26 00:13:21 config.py:685] Async output processing is not supported on the current platform type cpu.
WARNING 02-26 00:13:21 cpu.py:78] Environment variable VLLM_CPU_KVCACHE_SPACE (GB) for CPU backend is not set, using 4 by default.
WARNING 02-26 00:13:21 cpu.py:99] uni is not supported on CPU, fallback to mp distributed executor backend.
INFO 02-26 00:13:21 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/Volumes/T7_YR/models/embedding/gme-Qwen2-VL-2B-Instruct', speculative_config=None, tokenizer='/Volumes/T7_YR/models/embedding/gme-Qwen2-VL-2B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=No

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


WARNING 02-26 00:13:23 utils.py:175] Unable to collect loaded parameters for module Qwen2ForEmbedding(
WARNING 02-26 00:13:23 utils.py:175]   (model): Qwen2Model(
WARNING 02-26 00:13:23 utils.py:175]     (embed_tokens): VocabParallelEmbedding(num_embeddings=151936, embedding_dim=1536, org_vocab_size=151936, num_embeddings_padded=151936, tp_size=1)
WARNING 02-26 00:13:23 utils.py:175]     (layers): ModuleList(
WARNING 02-26 00:13:23 utils.py:175]       (0-27): 28 x Qwen2DecoderLayer(
WARNING 02-26 00:13:23 utils.py:175]         (self_attn): Qwen2Attention(
WARNING 02-26 00:13:23 utils.py:175]           (qkv_proj): QKVParallelLinear(in_features=1536, output_features=2048, bias=True, tp_size=1, gather_output=False)
WARNING 02-26 00:13:23 utils.py:175]           (o_proj): RowParallelLinear(input_features=1536, output_features=1536, bias=False, tp_size=1, reduce_results=True)
WARNING 02-26 00:13:23 utils.py:175]           (rotary_emb): MRotaryEmbedding(head_size=128, rotary_dim=128, max_pos

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
llm.llm_engine.model_config.task

'embed'

In [28]:
# Prepare chat formatter
tokenizer = llm.get_tokenizer()
model_config = llm.llm_engine.get_model_config()
resolved_content_format = resolve_chat_template_content_format(
    None, "auto", tokenizer
)
print(resolved_content_format)

INFO 02-26 00:13:37 chat_utils.py:332] Detected the chat template content format to be 'openai'. You can set `--chat-template-content-format` to override this.
openai


In [29]:
def prepare_inputs(messages):
    conversation, mm_data = parse_chat_messages(
        messages,
        model_config,
        tokenizer,
        content_format=resolved_content_format
    )
    prompt_data = apply_hf_chat_template(
        tokenizer,
        conversation=conversation,
        chat_template=None,
        add_generation_prompt=False,
        continue_final_message=False,
        tools=None
    )
    return prompt_data, mm_data

def embed(messages):
    prompt_data, mm_data = prepare_inputs(messages)
    outputs = llm.embed(
        {
            "prompt": prompt_data,
            "multi_modal_data": mm_data
        }
    )
    print(prompt_data,)
    print(outputs[0].outputs.embedding)
    embedding = outputs[0].outputs.embedding
    return embedding

In [30]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What is this car?"
            }
        ]
    }
]

embedding = embed(messages)
print(len(embedding))

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.26it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is this car?<|im_end|>

[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [31]:
embedding[:5]

[nan, nan, nan, nan, nan]